In [ ]:
!pip install asent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.7 MB/s eta 0:00:00


In [ ]:
import spacy
import asent
from nltk import word_tokenize, pos_tag
import nltk
import pandas as pd
import re
import string
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from tqdm import tqdm, tqdm_notebook
from tqdm._tqdm_notebook import tqdm_notebook

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tqdm_notebook.pandas()

<ipython-input-4-75bddb31632e>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
df_openwebtext10 = pd.read_csv('datasets/openwebtext_10k_sent_clean.csv')
df_openwebtext10

,text
0,A magazine supplement with an image of Adolf H...
1,"No law bans “ Mein Kampf ” in Germany , but th..."
2,The city that was the center of Adolf Hitler ’...
3,"What it does n’t have , nor has it since 1945 ..."
4,The latest attempt to publish excerpts fizzled...
...,...
378847,"After having bought some Imperial officers , h..."
378848,He came back to a depopulated and devastated R...
378849,Subsequently the Pope followed a policy of sub...
378850,Pope Clement VII is remembered for having orde...


In [ ]:
def length_symbols(sentence):
  return len(sentence)

In [ ]:
def length_words(sentence):
  return len(sentence.split())

In [ ]:
# upos part-of-speech tagging

def part_of_speech(sentence):
  pos = []
  doc = nlp(sentence)
  for token in doc:
    pos.append(token.pos_)
  return pos

In [ ]:
# list of entities: https://towardsdatascience.com/explorations-in-named-entity-recognition-and-was-eleanor-roosevelt-right-671271117218

def named_entity_recogniton(sentence):
  ner = {}
  doc = nlp(sentence)
  for ent in doc.ents:
    ner[ent.text] = ent.label_
  return ner

In [ ]:
def dependency_parsing(sentence):
  dep = {}
  doc = nlp(sentence)
  for token in doc:
    dep[token.text] = {'Dep': token.dep_, 'Head text': token.head.text}
  return dep

In [ ]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
def sentiment_analysis(sentence):
    try:
      sentence = preprocess(sentence)
      encoded_input = tokenizer(sentence, return_tensors='pt')
      output = model(**encoded_input)
      scores = output[0][0].detach().numpy()
      scores = softmax(scores) 
      return np.argmax(scores)
    except:
      return "Error"

In [ ]:
# https://stackoverflow.com/questions/30016904/determining-tense-of-a-sentence-python

def tense_detect(sentence):
  text = word_tokenize(sentence)
  tagged = pos_tag(text)
    
  verb_tags = ['MD','MDF',
              'BE','BEG','BEN','BED','BEDZ','BEZ','BEM','BER',
              'DO','DOD','DOZ',
              'HV','HVG','HVN','HVD','HVZ',
              'VB','VBG','VBN','VBD','VBZ',
              'SH',
              'TO',
              'JJ' # maybe?
              ]
    
  verb_phrase = []
  for item in tagged:
    if item[1] in verb_tags:
        verb_phrase.append(item)
  if (verb_phrase == []):
    return({'Not detected'})

  grammar = r"""
    future:                    {<MD><VB><VBN><VBG>}
    future:                    {<MD><VB><VBG>}
    future:                    {<MD><VB><VBN>}
    past:                      {<VBD><VBN><VBG>}
    present:                   {<VBP|VBZ><VBN><VBG>}
    future:                    {<MD><VB>}
    past:                      {<VBD><VBG>}
    past:                      {<VBD><VBN>}
    present:                   {<VBZ|VBP><VBG>}
    present:                   {<VBZ|VBP><VBN>}
    past:                      {<VBD>}
    present:                   {<VBZ>|<VBP>}
  """

  cp = nltk.RegexpParser(grammar)
  result = cp.parse(verb_phrase)
#  display(result)    
                      
  tenses_set = set()
  for node in result:
    if type(node) is nltk.tree.Tree:
        tenses_set.add(node.label())
  
  return tenses_set

In [ ]:
def add_all_attributes(df):
  df['Number of symbols'] = df['text'].progress_apply(length_symbols)
  print('Number of symbols added')
  df['Number of words'] = df['text'].progress_apply(length_words)
  print('Number of words added')
  nlp = spacy.load("en_core_web_sm")
  df['Parts of speech'] = df['text'].progress_apply(part_of_speech)
  print('Parts of speech added')
#  df['Named entity recognition'] = df['text'].apply(named_entity_recogniton)
#  print('Named entity recognition added')
#  df['Dependency parsing'] = df['text'].apply(dependency_parsing)
#  print('Dependency parsing added')
  df['Sentiment analysis'] = df['text'].progress_apply(sentiment_analysis)
  print('Sentiment analysis added') 
  df['Tense'] = df['text'].progress_apply(tense_detect)
  print('Tense added')
  return df

In [ ]:
df_openwebtext10 = add_all_attributes(df_openwebtext10)

In [ ]:
df_openwebtext10.to_csv('datasets/openwebtext_10k_sent_with_attributes.csv', index=False)